In [1]:
import rich
from neomodel import db

from mdmodels.graph import connect_to_neo4j, generate_neomodel

In [2]:
# Step 1: Connect to Neo4J
connect_to_neo4j(
    host="localhost",
    user="neo4j",
    password="12345678",
    port=7687
)

# Clear the database
db.cypher_query("MATCH (n) DETACH DELETE n")

([], [])

In [3]:
# Step 2: Generate neomodel types
types = generate_neomodel(path="model.md")
rich.print(types)

/Users/janrange/Documents/playground/sdrdm/mdmodels/mdmodels/graph/create.py:96: UserWarning: Attribute 'id' is reserved and will be renamed to 'id_' to avoid conflicts.
  warnings.warn(


{
    'ChemicalProject': <class 'neomodel.sync_.core.ChemicalProject'>,
    'Molecule': <class 'neomodel.sync_.core.Molecule'>,
    'Experiment': <class 'neomodel.sync_.core.Experiment'>,
    'Concentration': <class 'neomodel.sync_.core.Concentration'>,
    'Reaction': <class 'neomodel.sync_.core.Reaction'>,
    'Element': <class 'neomodel.sync_.core.Element'>,
    'UnitDefinition': <class 'neomodel.sync_.core.UnitDefinition'>,
    'BaseUnit': <class 'neomodel.sync_.core.BaseUnit'>
}

In [4]:
# Step 3: Create a new node
Molecule = types["Molecule"]
Reaction = types["Reaction"]

reaction = Reaction(name="Reaction").save()
substrate = Molecule(identifier="Substrate", name="Substrate").save()
product = Molecule(identifier="Product", name="Product").save()

In [5]:
# Step 4: Create a relationship
reaction.educts.connect(substrate)
reaction.products.connect(product)

True

In [6]:
# Step 4: Add out of schema relationship
reaction_2 = Reaction(name="Related Reaction").save()

reaction.dyn_connect(reaction_2, "RELATED", {"has_same_kinetics": True})

In [10]:
# Step 5: Query the database
reaction = Reaction.nodes.first()

# Display all relationships (in schema)
rich.print(reaction.educts.all())

# Display all relationships (including out of schema)
rich.print(reaction.get_relationships())

[
    <Molecule: {'id_': None, 'name': 'Substrate', 'formula': None, 'element_id_property': 
'4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:1'}>
]

[
    (
        <Reaction: {'id_': None, 'name': 'Reaction', 'element_id_property': 
'4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:0'}>,
        <Relationship element_id='5:886c4171-80d6-4dd4-ad0b-a1f24ac30557:0' nodes=(<Node 
element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:0' labels=frozenset() properties={}>, <Node 
element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:1' labels=frozenset({'Molecule'}) properties={'name': 
'Substrate'}>) type='CONSUMES' properties={}>,
        <Node element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:1' labels=frozenset({'Molecule'}) 
properties={'name': 'Substrate'}>
    ),
    (
        <Reaction: {'id_': None, 'name': 'Reaction', 'element_id_property': 
'4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:0'}>,
        <Relationship element_id='5:886c4171-80d6-4dd4-ad0b-a1f24ac30557:1' nodes=(<Node 
element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:0' labels=frozenset() properties={}>, <Node 
element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:2' labels=frozenset({'Molecule'}) properties={'name': 
'Product'}>) type='PRODUCES' properties={}>,
        <Node element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:2' labels=frozenset({'Molecule'}) 
properties={'name': 'Product'}>
    ),
    (
        <Reaction: {'id_': None, 'name': 'Reaction', 'element_id_property': 
'4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:0'}>,
        <Relationship element_id='5:886c4171-80d6-4dd4-ad0b-a1f24ac30557:2' nodes=(<Node 
element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:0' labels=frozenset() properties={}>, <Node 
element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:3' labels=frozenset({'Reaction'}) properties={'name': 'Related 
Reaction'}>) type='RELATED' properties={'has_same_kinetics': True}>,
        <Node element_id='4:886c4171-80d6-4dd4-ad0b-a1f24ac30557:3' labels=frozenset({'Reaction'}) 
properties={'name': 'Related Reaction'}>
    )
]